In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reading train and test data 

In [ ]:
train_df=pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/train.csv")
test_df=pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/test.csv")
print("train_df shape : ",train_df.shape)
print("test_df shape : ",test_df.shape)

Lets print sample data of train and test

In [ ]:
print("Train Sample Data")
train_df.head()


In [ ]:

print("Test Sample Data")
test_df.head()

Info of Train and Test data

In [ ]:
train_df.info()

In [ ]:
test_df.info()

From sample data and info observerd that in other than id_code all are float values. 

Check of missing data

In [ ]:
missing_columns_train=train_df.columns[train_df.isnull().any()].to_list()
missing_columns_test=test_df.columns[test_df.isnull().any()].to_list()
print("train missing data columns: ",missing_columns_train)
print("test missing data columns: ",missing_columns_test)

Both train and test has no missing or null values

Running some models on raw train data for understanding.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:

X=train_df.drop(['target','ID_code'],axis=1)
y=train_df['target']

In [ ]:
X1=train_df.drop(['ID_code'],axis=1)

In [ ]:
from sklearn.model_selection import StratifiedKFold

def run(splits):
    X1['kfold']=-1
    skfold=StratifiedKFold(splits)
    for index,(train_,valid_) in enumerate(skfold.split(X1,X1['target'])):
        X1.loc[valid_,'kfold']=index
      
    X1.to_csv('./stratified_data.csv',index=False)
run(10)

In [ ]:
def run(fold):
    df=pd.read_csv('./stratified_data.csv')
    #list of numerical columns
    #num_cols=['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week']
    #df.drop(num_cols,inplace=True,axis=1)
    #lbl_enc=LabelEncoder()
    #df['target']=lbl_enc.fit_transform(df['target'])
    features=[feat for feat in df.columns if feat not in ['target','kfold']]
    """for feat in features:
        if feat not in num_cols:
            df[feat]=df[feat].astype(str).fillna('NONE')
        
    for feat in features:
        if feat not in num_cols:
            lbl=LabelEncoder()
            df[feat]=lbl.fit_transform(df[feat])"""
    
    X_train=df[df['kfold']!=fold][features].reset_index(drop=True)
    X_test=df[df['kfold']==fold][features].reset_index(drop=True)
    y_train=df[df['kfold']!=fold]['target']
    y_test=df[df['kfold']==fold]['target']
    model = LogisticRegression(class_weight={0:0.9,1:0.1})
    model.fit(X_train,y_train)
    predictions=model.predict_proba(X_test)[:,1]
    auc=roc_auc_score(y_test,predictions)
    print(f'fold={fold} and AUC={auc}')


In [ ]:
run(0)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0,stratify=y)

In [ ]:
train_df['target'].value_counts()

In [ ]:
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
c_range=[0.001,0.01,0.1,1,10,100,1000]
tol_range=[0.00001,0.0001,0.001]
max_iter_range=list(range(100,1100,100))
distributions = dict(C=c_range,max_iter=max_iter_range,tol=tol_range)
log_R = LogisticRegression(class_weight={0:0.9,1:0.1})
clf = RandomizedSearchCV(log_R, distributions, random_state=0,cv=10,scoring='roc_auc',n_jobs=-1)
clf.fit(X_train, y_train)
print("search.best_params_ ",clf.best_params_)
print("search.best_score_ ",clf.best_score_)

In [ ]:
train_predict=clf.best_estimator_.predict_proba(X_train)[::,1]
test_predict=clf.best_estimator_.predict_proba(X_test)[::,1]
train_auc=roc_auc_score(y_train, train_predict)
test_auc=roc_auc_score(y_test, test_predict)
print("train_auc :",train_auc)
print("test_auc :",test_auc)

In [ ]:
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in range(df_test.shape[1]):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print(len(real_samples_indexes))
print(len(synthetic_samples_indexes))

In [ ]:
X_test_kaggle=test_df.drop(['ID_code'],axis=1)

In [ ]:
X_test_kaggle=scaler.transform(X_test_kaggle)

In [ ]:

kaggle_test_predict=clf.best_estimator_.predict_proba(X_test_kaggle)[::,1]

In [ ]:
preds_test=[1 if i>0.5 else 0 for i in kaggle_test_predict]


In [ ]:
# Save test predictions to file
output = pd.DataFrame({'ID_code': test_df['ID_code'].to_list(),
                       'target': kaggle_test_predict})
output.to_csv('./submission.csv', index=False)

In [ ]:
out=pd.read_csv("./submission.csv")
out

In [ ]:
train_predict[::,1]


In [ ]:
SGD=SGDClassifier()
alpha_range=[0.0001,0.001,0.01,0.1,1,10,100]
pen=['l1','l2']
distributions = dict(alpha=alpha_range,penalty=pen)

clf = RandomizedSearchCV(SGD, distributions, random_state=0,cv=10,scoring='roc_auc',n_jobs=-1)
search = clf.fit(X_train, y_train)
print("search.best_params_ ",search.best_params_)
print("search.best_score_ ",search.best_score_)

In [ ]:
SGD = SGDClassifier(penalty='l2',alpha=0.0001)
SGD.fit(X_train,y_train)
train_predict=log_R.predict(X_train)
test_predict=log_R.predict(X_test)
train_auc=roc_auc_score(y_train, train_predict)
test_auc=roc_auc_score(y_test, test_predict)
print("train_auc :",train_auc)
print("test_auc :",test_auc)

In [ ]:
RF=RandomForestClassifier()
n_estimators_range=[10,100,200,300,500,1000]
pen=['l1','l2']
distributions = dict(n_estimators=n_estimators_range)

clf = RandomizedSearchCV(RF, distributions, random_state=0,cv=10,scoring='roc_auc',n_jobs=-1)
search = clf.fit(X_train, y_train)
print("search.best_params_ ",search.best_params_)
print("search.best_score_ ",search.best_score_)